### 데이터 로딩

In [1]:
import pandas as pd
import numpy as np



C:\ana3\envs\t_f2.2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data = pd.read_csv("C:/Users/김건우/Desktop/UNIST/4학년 1학기/프로젝트랩/data/울산과학기술원_20.4월.txt", encoding='euc-kr')

In [135]:
temp = data

In [166]:
data = pd.read_csv("C:/Users/김건우/Desktop/UNIST/4학년 1학기/프로젝트랩/data/data_19_0106.csv")
temp = data[data['기준년월']==201902]

### Dask

In [1]:
import dask.dataframe as dd
import dask.array as da
import pandas as pd
import numpy as np

df_1 = dd.read_csv("C:/Users/김건우/Desktop/UNIST/4학년 1학기/프로젝트랩/data/data_19_0106.csv")
df_2 = dd.read_csv('C:/Users/김건우/Desktop/UNIST/4학년 1학기/프로젝트랩/data/data_19_0712.csv')
df_3 = dd.read_csv("C:/Users/김건우/Desktop/UNIST/4학년 1학기/프로젝트랩/data/data_20_0103.csv")
df_4 = dd.read_csv("data_april.csv")

data = [df_1, df_2, df_3, df_4]
x = da.concatenate(data, axis=0)

dask_columns = ['index','기준년월', '고객유형', '가맹점소재지1', '가맹점소재지2', '가맹점소재지3', '업종대분류명', '업종중분류명',
        '업종소분류명', '성별', '연령대별', '가구생애주기별', '연평균소득추정', '이용금액', '이용건수']

tmp_df = dd.io.from_dask_array(x,columns = dask_columns)

C:\ana3\envs\t_f2.2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
total_data = tmp_df.to_csv("total_data_dask.csv",single_file = True)

In [6]:
data_all = pd.read_csv('total_data_dask.csv')

In [ ]:
data_all = data_all.drop([data_all.columns[0],data_all.columns[1]],axis=1)

### new 업종분류

In [196]:
tmp = temp  # 월벼로 들어가는 데이터

In [111]:
'''tmp = pd.read_csv('C:/Users/김건우/Desktop/UNIST/4학년 1학기/프로젝트랩/data/울산과학기술원_20.4월.txt', 
                   engine='python', encoding='euc-kr')'''

In [197]:
middle_category_money = tmp['업종중분류명'] == '유통업영리'
middle_category_not_money = tmp['업종중분류명'] == '유통업비영리'

large_category_life = tmp['업종대분류명'] == '생활'
large_category_shopping = tmp['업종대분류명'] == '쇼핑'

life_not_money_ = tmp[middle_category_not_money&large_category_life]
life_money_ = tmp[middle_category_money&large_category_life]
shopping_money_ = tmp[middle_category_money&large_category_shopping]

In [198]:
# 쇼핑 유통업영리 이용건수 총 합 -> 대형 판매에 들어가는 일반백화점, 대형할인점, 면세점만 
split_shopping_money_1 = shopping_money_['업종소분류명'].str.contains('대형할인점|일반백화점|면세점')
shopping_money_1 = shopping_money_[split_shopping_money_1]
shopping_money_1_sum = shopping_money_1.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

# 쇼핑 유통업영리 이용건수 총 합 -> 전자상거래에 들어가는 통신판매, 홈쇼핑, PG상품권 
split_shopping_money_2 = shopping_money_['업종소분류명'].str.contains('통신판매|홈쇼핑|PG상품권')
shopping_money_2 = shopping_money_[split_shopping_money_2]
shopping_money_2_sum = shopping_money_1.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

# 생활 유통업비영리 이용건수 총 합 -> 종합소매에 들어가는 all
life_not_money_sum = life_not_money_.groupby(['업종소분류명'],as_index = False).sum()['이용건수']


# 생활 유통업영리 이용건수 총합 -> 종합소매에 들어가는 슈퍼마켓 편의점, 연쇄점,복지, 매점, 농축수산가공품
split_life_money_1 = life_money_['업종소분류명'].str.contains('슈퍼마켓|편의점|연쇄점|복지매점|농축수산가공품')
life_money_1 = life_money_[split_life_money_1]
life_money_1_sum = life_money_1.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

# 생활 유통업영리 이용건수 총합 -> 전자상거래에 들어가는 인터넷종합Mall|상품권|인터넷PG|복지매점|전자상거래상품권
split_life_money_2 = life_money_['업종소분류명'].str.contains('인터넷종합Mall|상품권|인터넷PG|전자상거래상품권|인터넷Mall')
life_money_2 = life_money_[split_life_money_2]
life_money_2_sum = life_money_2.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

In [199]:
tmp_data_1904 = tmp[['업종대분류명','업종중분류명','업종소분류명','이용건수']]
tmp_df = tmp_data_1904.groupby(['업종대분류명', '업종중분류명','업종소분류명'], as_index=False).sum()
tmp_df.columns = ['Large','Middle','Small','Count']
tmp_df = pd.DataFrame(tmp_df)
'''Large_sum = tmp_df.groupby(['Large'],as_index = False).sum()['Count']
for i in range(len(Large_sum)):
    a = {tmp_df['Large'].unique()[i] : Large_sum[i]}
    print(a)'''

"Large_sum = tmp_df.groupby(['Large'],as_index = False).sum()['Count']\nfor i in range(len(Large_sum)):\n    a = {tmp_df['Large'].unique()[i] : Large_sum[i]}\n    print(a)"

In [200]:
classified_df =  pd.read_excel('column_categorize_bc_card.xlsx')
concat_name = pd.DataFrame(classified_df['Large'] + '-' + classified_df['Middle'])
concat_name = concat_name.dropna()
new_classified = pd.concat([concat_name,classified_df['분류'],classified_df['total count']],axis=1)
new_classified.columns = ['category','num','total count']
new_category_name = list(classified_df['종목'][:16])
new_category_name += '대형판매', '종합소매', '전자상거래'
num_name = list(classified_df['기준'][:16])

In [201]:
for i in range(len(new_classified)):
    if new_classified['num'][i] == 'x':
        continue
    elif int(new_classified['num'][i]) == 1:
        new_classified.at[i,'new_category_name'] = new_category_name[0]
        
    elif int(new_classified['num'][i]) == 2:
        new_classified.at[i,'new_category_name'] = new_category_name[1]
        
    elif int(new_classified['num'][i]) == 3:
        new_classified.at[i,'new_category_name'] = new_category_name[2]
        
    elif int(new_classified['num'][i]) == 4:
        new_classified.at[i,'new_category_name'] = new_category_name[3]
    
    elif int(new_classified['num'][i]) == 5:
        new_classified.at[i,'new_category_name'] = new_category_name[4]
    
    elif int(new_classified['num'][i]) == 6:
        new_classified.at[i,'new_category_name'] = new_category_name[5]
        
    elif int(new_classified['num'][i]) == 7:
        new_classified.at[i,'new_category_name'] = new_category_name[6]
        
    elif int(new_classified['num'][i]) == 8:
        new_classified.at[i,'new_category_name'] = new_category_name[7]
        
    elif int(new_classified['num'][i]) == 9:
        new_classified.at[i,'new_category_name'] = new_category_name[8]
        
    elif int(new_classified['num'][i]) == 10:
        new_classified.at[i,'new_category_name'] = new_category_name[9]    
        
    elif int(new_classified['num'][i]) == 11:
        new_classified.at[i,'new_category_name'] = new_category_name[10]
        
    elif int(new_classified['num'][i]) == 12:
        new_classified.at[i,'new_category_name'] = new_category_name[11]
        
    elif int(new_classified['num'][i]) == 13:
        new_classified.at[i,'new_category_name'] = new_category_name[12]
    
    else:
        continue


In [202]:
added_category = {
    'category' : ['일반백화점','슈퍼마켓','홈쇼핑'],
    'num' : ['x','x','x'],
    'total count' : [shopping_money_1_sum.sum(), 
                     life_not_money_sum.sum()+life_money_1_sum.sum(),
                    life_money_2_sum.sum()+shopping_money_2_sum.sum()],
    'new_category_name' : ['대형 판매', '종합소매', '전자상거래']
}
added_df = pd.DataFrame(added_category)

In [203]:
new_classified = new_classified.dropna()
new_classified = pd.concat([new_classified,added_df],axis=0)
new_classified = new_classified.drop(['num'],axis=1)
new_classified = new_classified.reset_index()
new_classified = new_classified.drop(['index'],axis=1)

### 임시 pandas 분류 -> 사용 x

### 4.29 추가한거

In [ ]:
z = tmp[:100]
z['category'] = z['업종대분류명']+ '-' + z['업종중분류명']
z = z[['category','업종소분류명']]
aaa_list = []

for i in tqdm(range(len(z)),'진행도'):
    for k in range(len(new_classified)):
        if z['category'][i] == new_classified['category'][k]:
            z.at[i,'new_category_name'] = new_classified['new_category_name'].iloc[k]
            aaa_list.append(new_classified['new_category_name'].iloc[k])
            
        elif z['category'][i] == '생활-유통업영리':
            if z['업종소분류명'][i] in life_money_1['업종소분류명'].unique():
                z.at[i,'new_category_name'] = '종합소매'
                aaa_list.append('종합소매')
            elif z['업종소분류명'][i] in life_money_2['업종소분류명'].unique():
                z.at[i,'new_category_name'] = '전자상거래'
                aaa_list.append('전자상거래')
                
        elif z['category'][i] == '생활-유통업비영리':
            z.at[i,'new_category_name'] = '종합소매'
            aaa_list.append('종합소매')
            
        elif z['category'][i] == '쇼핑-유통업영리':
            if z['업종소분류명'][i] in shopping_money_1['업종소분류명'].unique():
                z.at[i,'new_category_name'] = '대형 판매'
                aaa_list.append('대형 판매')
            elif z['업종소분류명'][i] in shopping_money_2['업종소분류명'].unique():
                z.at[i,'new_category_name'] = '전자상거래'
                aaa_list.append('전자상거래')

In [204]:
### str -> int mapping

def change_category (df):
    new_list = []
    a = list(df.category.unique())
    for i in range(len(a)):
        new_list.append(i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic

def change_upjong (df):
    new_list = []
    a = list(df.업종소분류명.unique())
    for i in range(len(a)):
        new_list.append(100+i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic

In [205]:
from tqdm import tqdm

In [206]:
z = tmp
z['category'] = z['업종대분류명']+ '-' + z['업종중분류명']
z = z[['category','업종소분류명']]
z = z.to_numpy()

change_category = change_category(tmp)
change_upjong = change_upjong(tmp)

for i in tqdm(range(len(z)),'1번째'):
    z[i,0] = change_category[z[i,0]]
    
for i in tqdm(range(len(z)),'2번째'):
    z[i,1] = change_upjong[z[i,1]]

<ipython-input-206-45ae6d11ee09>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['category'] = z['업종대분류명']+ '-' + z['업종중분류명']
2번째: 100%|█████████████████████████████████████████████████████████████| 5340345/5340345 [00:08<00:00, 630687.07it/s]


In [208]:
change_category

{'생활-음식료품': 0,
 '생활-의료기관': 1,
 '생활-자동차정비/유지': 2,
 '쇼핑-신변잡화': 3,
 '유흥-휴게': 4,
 '음식-일반음식': 5,
 'T&E-숙박업': 6,
 '문화-학원': 7,
 '생활-연료판매': 8,
 '생활-유통업비영리': 9,
 '생활-유통업영리': 10,
 'T&E-레져업소': 11,
 '문화-문화/취미': 12,
 '유흥-보건/위생': 13,
 '기타-기타': 14,
 '내구재-건축/자재': 15,
 'T&E-레져용품': 16,
 '기타-용역서비스': 17,
 '문화-서적/문구': 18,
 '생활-보건/위생': 19,
 '생활-수리서비스': 20,
 '쇼핑-유통업영리': 21,
 '쇼핑-의류': 22,
 '음식-단란주점(음식)': 23,
 '내구재-농업': 24,
 '음식-유흥주점(음식)': 25,
 'T&E-회원제형태업소': 26,
 '내구재-가구': 27,
 '내구재-가전제품': 28,
 '문화-광학제품': 29,
 '생활-건강식품': 30,
 '생활-직물': 31,
 '내구재-주방용품': 32,
 '문화-회원제형태업소': 33,
 '생활-용역서비스': 34,
 '내구재-사무/통신기기': 35,
 'T&E-여행업': 36,
 '기타-회원제형태업소': 37,
 '생활-보험': 38,
 '내구재-자동차판매': 39,
 '기타-보건/위생': 40,
 '생활-회원제형태업소': 41}

In [210]:
for i in range(len(new_classified)):
    try:
        new_classified.category[i] = change_category[new_classified.category[i]]
    except:
        continue

<ipython-input-210-be6cf286e842>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_classified.category[i] = change_category[new_classified.category[i]]


In [211]:
new_classified = new_classified.drop(['total count'],axis=1)
new_classified = new_classified.to_numpy()
#new_classified

In [212]:
l_1 = []
l_2 = []
s_1 = []
s_2 = []

for i in list(life_money_1.업종소분류명.unique()):
    l_1.append(change_upjong[i])
    
# life money 2 4check
for i in list(life_money_2.업종소분류명.unique()):
    l_2.append(change_upjong[i])
    
for i in list(shopping_money_1.업종소분류명.unique()):
    s_1.append(change_upjong[i])
    
for i in list(shopping_money_2.업종소분류명.unique()):
    s_2.append(change_upjong[i])

In [213]:
tmp_list = []

for i in tqdm((range(len(z))),'진행도'):
    for k in range(len(new_classified)):   
        if z[i,0] == new_classified[k,0]:
            tmp_list.append(new_classified[k,1])
            
    if z[i,0] == 10:    # 생활 유통업영리
        if z[i,1] in l_1:     #생활 유통업영리 (소분류가 비전자)
            tmp_list.append('종합소매')
            continue
        
        #변경된 부분
        elif z[i,1] in l_2:    #생활 유통업영리 (소분류가 전자)
            tmp_list.append('전자상거래')
            continue

        else:
            tmp_list.append('예외처리')
            continue

    elif z[i,0] == 9:          # 생활 유통업비영리
        tmp_list.append('종합소매')
        continue

    elif z[i,0] == 21:           # 쇼핑 유통업영리
        if  z[i,1] in s_1:
            tmp_list.append('대형 판매')
            continue

        elif z[i,1] in s_2:
            tmp_list.append('전자상거래')
            continue

        else:
            tmp_list.append('예외처리')
            continue

진행도: 100%|█████████████████████████████████████████████████████████████| 5340345/5340345 [03:48<00:00, 23408.91it/s]


In [214]:
a = temp[temp['업종대분류명']=='생활']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='인터넷Mall']
len(c)

3577

In [215]:
a = temp[temp['업종대분류명']=='생활']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='인터넷종합Mall']
len(c)

6976

In [216]:
a = temp[temp['업종대분류명']=='생활']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='상품권']
len(c)

3411

In [217]:
a = temp[temp['업종대분류명']=='생활']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='인터넷PG']
len(c)

4806

In [218]:
a = temp[temp['업종대분류명']=='생활']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='전자상거래상품권']
len(c)

289

In [219]:
a = temp[temp['업종대분류명']=='쇼핑']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='통신판매']
len(c)

270

In [220]:
a = temp[temp['업종대분류명']=='쇼핑']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='홈쇼핑']
len(c)

946

In [221]:
a = temp[temp['업종대분류명']=='쇼핑']
b = a[a['업종중분류명']=='유통업영리']
c = b[b['업종소분류명']=='PG상품권']
len(c)

568

In [222]:
len(temp)

5340345

In [223]:
len(tmp_list)

5340345

In [224]:
category_df = pd.DataFrame(data=tmp_list,columns=['new_category'])
category_df.head()

,new_category
0,식료품
1,식료품
2,식료품
3,식료품
4,식료품


In [227]:
(unique, counts) = np.unique(tmp_list, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([['가구 및 전자 제품', '62977'],
       ['교육', '188142'],
       ['교통', '31710'],
       ['기타', '91043'],
       ['대형 판매', '170866'],
       ['보험', '3403'],
       ['서비스', '351864'],
       ['식료품', '575328'],
       ['여행 및 숙박', '391800'],
       ['예외처리', '9009'],
       ['오락 및 문화', '434268'],
       ['음식점', '990169'],
       ['의료', '582759'],
       ['의류 및 잡화', '202363'],
       ['자동차', '377708'],
       ['전자상거래', '20843'],
       ['종합소매', '856093']], dtype='<U21')

In [225]:
len(category_df.new_category.unique())

17

In [226]:
(category_df.new_category.unique())

array(['식료품', '의료', '자동차', '의류 및 잡화', '여행 및 숙박', '음식점', '교육', '종합소매',
       '오락 및 문화', '기타', '서비스', '대형 판매', '가구 및 전자 제품', '교통', '예외처리',
       '전자상거래', '보험'], dtype=object)